In [1]:
import importlib
import full_model

In [2]:
full_model.eng = full_model.start_matlab()


Starting MATLAB engine...
-------------------------
Elapsed: 19.53 sec


In [3]:
images_norm = full_model.loadmat("./saved/yaleB.mat")['X']
labels = full_model.loadmat("./saved/yaleB.mat")['Y'].reshape(-1)

## Try various Hyperparameter tuning methods
* [Scikit-Optimize](https://scikit-optimize.github.io/notebooks/hyperparameter-optimization.html)

In [4]:
importlib.reload(full_model)

<module 'full_model' from 'C:\\Users\\aleks_000\\Desktop\\Mentorship\\!GitHub\\full_model.py'>

In [65]:
from skopt.space import Real, Integer

space  = [Real(10**-2, 10**0, "log-uniform", name='lr_pretrain'),
          Real(10**-3, 10**-1, "log-uniform", name='lr'),
          Real(10**0, 10**2, "log-uniform", name='alpha1'),
          Integer(2, 100, name='maxIter1'),
          Real(10**0, 10**2, "log-uniform", name='alpha2'),
          Integer(2, 100, name='maxIter2'),]

In [66]:
from skopt.utils import use_named_args

@use_named_args(space)
def objective(**params):
    try:
        return 1-full_model.run_model(images_norm, labels, ep_prtr, ep, **params)[0]
    except:
        return 1

In [68]:
from skopt import gp_minimize
from skopt.plots import plot_convergence
import time

def optimize(iterations, epochs_pretrain, epochs):
    start_time = time.time()
    
    global ep_prtr, ep
    ep_prtr = epochs_pretrain
    ep = epochs
    res_gp = gp_minimize(objective, space, n_calls=iterations, random_state=0, verbose=True, n_jobs=-1)

    plot_convergence(res_gp)
    print("\n------------------")
    print("Best score: {0:.4f}".format(res_gp.fun))
    for i in range(10, iterations+1, 10):
        print("\t{0:d}: {1:.4f}".format(i, min(res_gp.func_vals[0:i])))
    print("Total time elapsed: {0:.2f} sec\n".format(time.time()-start_time))
    return res_gp.x

In [ ]:
# Pro-tip:
# Program taking too long?
# No problem
# ----------
# For a quick finish, delete the temp folder,
# wait for the code to finish, then create it again
# or
# Deny Admin permissions to temp.mat,
# wait for the code to finish, then remove the restrictions
res_gp = optimize(100, 101, 26)

In [ ]:
def reload(res, addtl_calls):
    return gp_minimize(objective, space,
                       n_calls=addtl_calls-len(),
                       n_random_starts=0,
                       x0=res.x_iters,
                       y0=res.func_vals,
                       random_state=0)